In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

from string import Template

In [ ]:
device = 'auto'
LLM_model = './custom_LLM_final'

# SOLAR prompt template
prompt_template = Template(
    """### System:
${system}

### User:
${user}

### Assistant:
""")

generation_config = GenerationConfig(
    max_new_tokens=512,
    )

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    LLM_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(LLM_model)

In [ ]:
user_inputs = [
    "면진장치에 사용되는 주요 기술은 무엇인가요?",
    "내진설계의 종류 좀 알려줘",
    "철골구조의 장점을 알려줘?",
    "철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?",
    "철골구조의 다양한 방식이 무엇인가요?",
]

In [ ]:
def test_prompt(system_input, user_inputs, generation_config):
    prompts = (prompt_template.substitute(system=system_input, user=user_input) for user_input in user_inputs)
    inputs = [tokenizer.encode(prompt, return_tensors="pt").to('cuda') for prompt in prompts]
    output_texts = []
    
    for user_input in inputs:
        outputs = model.generate(
            user_input,
            generation_config=generation_config,
        )
        output_text = tokenizer.batch_decode(outputs)[0]
        output_texts.append(output_text)

    return output_texts

In [ ]:
outputs = test_prompt(
    system_input="당신은 도배 전문가 입니다. 전문가의 입장으로 User의 질문에 답하세요.",
    user_inputs=user_inputs,
    generation_config=generation_config,
)

for output in outputs:
    print(output, '\n\n')